In [190]:
# validationf VeRITAS
# we validate how well veritas is doing when we cannot ask too many question
# Typically, we want to evaluate performance in conjunction with a CAT framework
# Conclusion: It seems CAT's question generation is not abest for Veritas
# we can ask similar number of questions, but get much better performance if we 
# generate the questiosn differently, namely questions that not only 
# take into account the goal of getting to diagnosis, but also detection of malingering

In [191]:
def selrandomq(jsondata,num):
    '''
    sel random num number of responses from a json
    '''
    from random import sample 
    subname=list(jsondata.keys())[0]
    keys=list(jsondata[subname].keys())
    return {subname: {k:jsondata[subname][k] for k in sample(keys,num)}}
#selrandomq(fulldata20[0],5)    

In [192]:
import sys
sys.path.append("../") 
from infer_veritas import  *
from truthfinder import *
import gzip

In [193]:
! pwd

/home/ishanu/ZED/Research/truthnet/veritas_test/ntb


In [194]:
CATTHETA=pd.read_csv('../response_jsons/cattheta713.csv',index_col=0)
with gzip.open('../response_jsons/validation_cat713.pkl.gz', 'rb') as filepath:
    cat713data = pickle.load(filepath)
Veritas_model_path='../veritas_models/veritas_ptsd.pkl.gz'    

Veritas_model_path='../veritas_models/veritas_001.pkl.gz'

In [195]:
with gzip.open(Veritas_model_path, 'rb') as filepath:
    TR= pickle.load(filepath)

In [197]:
TR

{'version': '0.0.1',
 'model': qnet.Qnet,
 'model_neg': qnet.Qnet,
 'problem': '',
 'dist_lower': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f0204989ff0>,
 'LOWER_THRESHOLD': 0.9154097135778224,
 'dist_veritas': <scipy.stats._distn_infrastructure.rv_continuous_frozen at 0x7f020498a950>,
 'VERITAS_THRESHOLD': 0.7766973222129768,
 'upper_scoretoprobability': <bound method processRoc.scoretoprobability of <zedstat.zedstat.processRoc object at 0x7f020498b850>>,
 'UPPER_THRESHOLD': 1.297963929289763,
 'AUC': (0.8575819672131147, 0.8657822325456725, 0.849381701880557)}

In [199]:
Vm=TR

In [200]:
CATTHRESHOLD=0
DX=(CATTHETA.theta<CATTHRESHOLD).values.astype(int)
DX.sum()

423

In [201]:
Rcat=reveal(cat713data,Vm,perturb=0,model_path=False)

ckeck passed


In [205]:
def malingering(lower_threshold,veritas,upper,C0=1,C1=0.76,C2=1.35):
    M=False
    if (lower_threshold < C0) and (upper > C2):
        M=True
    if (veritas > C1) and (upper > C2):
        M=True
    return int(M)


def malingeringCAT(lower_threshold,veritas,theta,C0=1,C1=0.76,C2=0):
    M=False
    if (lower_threshold < C0) and (theta > C2):
        M=True
    if (veritas > C1) and (theta > C2):
        M=True
    return int(M)

In [219]:
dxRcat=[i['score']>1.35 for i in Rcat[0]]
theta=[x[0] for x in pd.read_csv('../response_jsons/cattheta713.csv',index_col=0).values.astype(float)]
mgcat=[malingeringCAT(i['lower_threshold'],i['veritas'],j) for i,j in zip(Rcat[0],theta)]
mgRcat=[malingering(i['lower_threshold'],i['veritas'],i['score']) for i in Rcat[0]]
k=[i['lower_threshold'] for i in Rcat[0]]
v=[i['veritas'] for i in Rcat[0]]
s=[i['score'] for i in Rcat[0]]

dxcat=[x>0 for x in theta]


Dcat=pd.DataFrame([(i['subject_id'],malingering(i['lower_threshold'],i['veritas'],i['score'])) for i in Rcat[0]],columns=['sub','mgRcat'])
#print(Dcat[(Dcat.dx==1) & (Dcat.mgRcat==1)].index.size)
Dcat=Dcat.assign(theta=theta,mgcat=mgcat,k=k,v=v,s=s,dxRcat=dxRcat,dxcat=dxcat)#.corr()
Dcat.corr()

,mgRcat,theta,mgcat,k,v,s,dxRcat,dxcat
mgRcat,1.000000,-0.112392,-0.193858,-0.537014,-0.305197,0.696912,0.789156,-0.227536
theta,-0.112392,1.000000,0.724377,0.110626,-0.042609,0.008210,-0.166595,0.793376
mgcat,-0.193858,0.724377,1.000000,0.279723,0.250390,-0.183162,-0.290597,0.906683
k,-0.537014,0.110626,0.279723,1.000000,0.559244,-0.723635,-0.602481,0.283344
v,-0.305197,-0.042609,0.250390,0.559244,1.000000,-0.613250,-0.489902,0.150687
s,0.696912,0.008210,-0.183162,-0.723635,-0.613250,1.000000,0.826580,-0.193375
dxRcat,0.789156,-0.166595,-0.290597,-0.602481,-0.489902,0.826580,1.000000,-0.318177
dxcat,-0.227536,0.793376,0.906683,0.283344,0.150687,-0.193375,-0.318177,1.000000


In [220]:
Dcat.sum()

sub       CAT1001CAT1002CAT1003CAT1004CAT1005CAT1006CAT1...
mgRcat                                                  100
theta                                              -101.946
mgcat                                                   257
k                                               1019.673255
v                                                590.362601
s                                                657.997557
dxRcat                                                  148
dxcat                                                   290
dtype: object

In [212]:
Dcat

,dx,mgRcat,theta,mgcat,k,v,s,dxRcat
0,1,0,-0.012,0,1.620533,0.720000,1.030817,False
1,2,0,-0.013,0,1.199156,0.800000,1.236916,False
2,3,1,-0.077,0,1.404334,0.900000,1.513398,True
3,4,0,0.303,1,1.918270,0.920000,0.449951,False
4,5,0,0.043,1,1.454088,0.920000,0.757492,False
...,...,...,...,...,...,...,...,...
708,9,1,0.294,1,1.388919,0.840000,1.630983,True
709,0,0,0.463,1,1.024627,0.888889,1.328577,False
710,1,0,0.062,1,2.113787,0.920000,0.229213,False
711,2,0,0.081,1,1.072690,0.839485,1.308845,False
